In [2]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import randForest as rfr
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_curve, auc, roc_auc_score, f1_score
import joblib

In [3]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Importing and manipulating datasets

In [ ]:
sklearn.metrics.get_scorer_names()

In [ ]:
datasetBasin1.csv

In [4]:
# basin1CleanToTrain.csv
# original = pd.read_csv('basin1CleanToTrain.csv', index_col = None)
DS = pd.read_csv('datasetBasin1.csv', index_col = None)

In [5]:
DS.describe()

,percentage,disToRiv,TWI,TPI,FAcc,slope,elevation,x_coord,y_coord
count,2.493980e+06,2.493980e+06,2.493979e+06,2.493980e+06,2.493980e+06,2.479538e+06,2.493980e+06,2.493980e+06,2.493980e+06
mean,9.150033e-02,2.097987e+02,9.478135e+00,1.100693e-04,4.330615e+04,4.821367e+00,1.455343e+02,3.650845e+05,5.264240e+06
std,6.592287e-01,1.727702e+02,4.079174e+00,1.950585e-01,1.029387e+06,4.994698e+00,4.576501e+01,3.175800e+03,1.830620e+03
min,0.000000e+00,0.000000e+00,-9.252120e+00,-7.461320e+00,2.500000e+01,4.700000e-04,-3.808000e-02,3.586730e+05,5.260213e+06
25%,0.000000e+00,7.424622e+01,5.551585e+00,-6.195000e-02,8.772391e+01,1.600760e+00,1.159088e+02,3.623130e+05,5.262718e+06
50%,0.000000e+00,1.691892e+02,1.025464e+01,-2.100000e-04,2.851745e+02,3.192175e+00,1.483494e+02,3.650780e+05,5.264348e+06
75%,0.000000e+00,3.030264e+02,1.288554e+01,6.131000e-02,1.254889e+03,6.192805e+00,1.800543e+02,3.676380e+05,5.265788e+06
max,5.000000e+00,1.355775e+03,2.486353e+01,6.706970e+00,6.282632e+07,7.002943e+01,2.605691e+02,3.721230e+05,5.267983e+06


## balanced sampling

In [19]:
## Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit
X,Y = ms.importDataSet('datasetBasin1.csv', 'percentage')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index.size, "TEST:", test_index)
    X_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]

TRAIN: 1995184 TEST: [ 756851  359082 1003874 ... 1000311 1257022 1105391]


In [20]:
print( y_train.value_counts())
print(y_test.value_counts())

0    1952867
5      35061
1       7256
Name: percentage, dtype: int64
0    488217
5      8765
1      1814
Name: percentage, dtype: int64


In [24]:
## This proportions ar

totalTrain = sum([1952867, 35061,7256])
totalValidation = sum([488217,8765,1814])
print("Balance summary of train dataset")
print(f"Class 1% : {1952867/totalTrain}, Class 1%: {7256/totalTrain}, Class 5%: {35061/totalTrain}")
print(f"Class 1% : {488217/totalValidation}, Class 1%: {1814/totalValidation}, Class 5%: {8765/totalValidation}")

Balance summary of train dataset
Class 1% : 0.9787904273490565, Class 1%: 0.003636757311606348, Class 5%: 0.017572815339337123
Class 1% : 0.9787909285559627, Class 1%: 0.003636757311606348, Class 5%: 0.017572314132430895


In [ ]:
X,Y = ms.importDataSet('basin1_FaccNorm.csv', 'percentage')
x_train,x_validation,y_train, y_validation = train_test_split(X,Y, test_size = 0.2) 

In [ ]:
x_validation['percentage'] = y_validation
print(x_validation.head())

In [ ]:
x_validation.to_csv('basin1Validation.csv', index = None)

In [ ]:
x_train['percentage'] = y_train
print(x_train.head())

In [ ]:
x_train.to_csv('basin1Train.csv', index = None)

In [ ]:
RFClassifier = rfr.implementRandomForestCalssifier()

In [ ]:
model = rfr.loadModel('./bestModels/21-46-40 Classifier/2207232146.pkl')

In [ ]:
print(model)

# Data description and visualization

In [ ]:
### FAcc vs Labels
targets = DS['percentage']
FAcc = original['FAcc']
FAcc_norm = DS['FAcc_norm']
fig, axs = plt.subplots(1, 2, figsize=(13,4), sharey=True)
fig.text(-0.02, 0.5, 'labels', va='center', rotation='vertical')
fig.text(0.5, 1, 'Flow accumulation vs labels distribution', ha ='center')
axs[0].scatter(FAcc,targets)
# axs[0].set_title("Facc")
axs[0].set(xlabel='a) Flow Accumulation')
axs[1].scatter(FAcc_norm,targets)
# axs[1].set_title("FAcc_norm")
axs[1].set(xlabel='b) Flow Accumulation estandardized')
plt.rcParams['font.size'] = '20'
fig.tight_layout()


In [ ]:
##3 Al features vs labels
# 'disToRiv', 'TWI', 'TPI', 'slope', 'elevation',
targets = DS['percentage']
# targets = np.where( targets == 5, 2,targets)

E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc_norm']
S = DS['disToRiv']
TWI = DS['TWI']
TPI = DS['TPI']

fig, axs = plt.subplots(3, 2, figsize=(13, 8), sharey=True)
fig.supylabel('Labels')
plt.rcParams['font.size'] = '15'
plt.yticks([0,1,5])
axs[0, 0].scatter(E,targets)
axs[0, 0].set_title("Elevation")
axs[1, 0].scatter(slope,targets)
axs[1, 0].set_title("Slope")
axs[2, 0].scatter(FAcc,targets)
axs[2, 0].set_title("Flow accumulation")
axs[0, 1].scatter(S,targets)
axs[0, 1].set_title("Distance to river")
axs[1, 1].scatter(TWI,targets)
axs[1, 1].set_title("TWI")
axs[2, 1].scatter(TPI,targets)
axs[2, 1].set_title('TPI')
fig.tight_layout()


In [ ]:
print(DS.head())
#  Return a dataset with the rows corresponding to the index where condition in DS.columName is valid. 
dsArray = DS[DS.percentage != 0] print(dsArray.head())
print(dsArray.head()) 

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(DS, hue = 'percentage', diag_kind = 'kde', 
             plot_kws = {'alpha': 0.8, 's': 100},
             height = 4, corner=True, palette = "Set2")# vars = ['life_exp', 'log_pop', 'log_gdp_per_cap'],

# sns.pairplot(DS, hue="percentage")

In [ ]:
####. Covariance Matrix
matrix = DS.corr().round(2)
sns.heatmap(matrix, annot=True)
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(criterion='entropy', random_state = 50)
x_train,y_train = ms.importDataSet('basin1Train.csv', 'percentage')
classifier = OneVsRestClassifier(estimator).fit(x_train,y_train)


In [ ]:
# classifier = rfr.loadModel('./outputs/2022-08-02/11-01-47/2208021101.pkl')
x_test,y_test = ms.importDataSet('basin1Validation.csv', 'percentage')
y_prob = classifier.predict_proba(x_test)
#print(np.unique(y_prob))

In [ ]:
rfr.plot_ROC_AUC_OneVsRest(classifier, x_test, y_test)